<a href="https://colab.research.google.com/github/shivavsrivastava/Algorithms/blob/main/Course3_W1_Scheduling_MST_Prims.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scheduling Problems & Prim's MST


In [ ]:
import numpy as np
import random
import urllib3
import math
from collections import defaultdict
import time
import heapq
from heapq import heappush, heappop, heapify
import pandas as pd
import sys
from itertools import count

## Programming Assignment: Part 1


This file describes a set of jobs with positive and integral weights and lengths.  It has the format

[number_of_jobs]

[job_1_weight] [job_1_length]

[job_2_weight] [job_2_length]

...

For example, the third line of the file is "74 59", indicating that the second job has weight 74 and length 59.

You should NOT assume that edge weights or lengths are distinct.

Your task in this problem is to run the greedy algorithm that schedules jobs in decreasing order of the difference (weight - length).  Recall from lecture that this algorithm is not always optimal.  IMPORTANT: if two jobs have equal difference (weight - length), you should schedule the job with higher weight first.  Beware: if you break ties in a different way, you are likely to get the wrong answer.  You should report the sum of weighted completion times of the resulting schedule --- a positive integer --- in the box below.

ADVICE: If you get the wrong answer, try out some small test cases to debug your algorithm (and post your test cases to the discussion forum).

In [ ]:
df_jobs = pd.read_table('https://d3c33hcgiwev3.cloudfront.net/_642c2ce8f3abe387bdff636d708cdb26_jobs.txt?Expires=1713657600&Signature=FDXI5Ue-ZifNu0aGgxlGBwmlg7ZieAG78k7A8lOW1OOGoQK~04tMQpyT8PcHUcW6aRUZ~axNZo0cHJ1ZVjQB2j~2MZnx9Yqay3te4FMZM0c32xke53JYA00dt29F-WMW0zIo1FuQavI-MT8QKKj2wYalIEr-z9aztpa6RIvAuas_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A',header=None)

In [ ]:
num_jobs = df_jobs[0].iloc[0]
print(num_jobs)


In [ ]:
df_jobs = df_jobs.drop(df_jobs.index[0])
df_jobs.head()

In [ ]:
jobsArray = df_jobs[0].to_list()
jobsArray[:5]

In [ ]:
jobsDict = defaultdict(list)
for i in range(int(num_jobs)):
  job = jobsArray[i].strip().split()
  #print(job)
  # convert string to integer
  res = [eval(x) for x in job]
  #print(res)
  jobsDict[i].append(res[0])
  jobsDict[i].append(res[1])
  jobsDict[i].append(res[0] - res[1])
print(jobsDict)

In [ ]:
## I know that my dictionary has a list of values
def reverse_sort_dict_by_value(dict1):
    """Sorts a dictionary by value in descending order, breaking ties by one of the values in the list."""

    # Create a list of tuples, where each tuple contains a key-value pair.
    items = list(dict1.items())
    print("Unsorted dictionary")
    print(items)

    # Sort the list of tuples by value in descending order and on a tie look at weights
    items.sort(key=lambda x: (x[1][2],x[1][0]) , reverse=True)
    print("Sorted dictionary")
    print(items)

    return items

In [ ]:
#reverse_sort_dict_by_value(jobsDict)

#### Sorted dictionary is ready, now get the total completion time

In [ ]:
total_completion_time = 0
completion_time_so_far = 0
for item in reverse_sort_dict_by_value(jobsDict):
  completion_time_so_far += item[1][1]
  total_completion_time += (completion_time_so_far*item[1][0])
print(total_completion_time)

Answer is: 69119377652

## Programming Assignment: 2

For this problem, use the same data set as in the previous problem.

Your task now is to run the greedy algorithm that schedules jobs (optimally) in decreasing order of the ratio (weight/length).  In this algorithm, it does not matter how you break ties.  You should report the sum of weighted completion times of the resulting schedule --- a positive integer --- in the box below.

In [ ]:
jobsDict2 = defaultdict(list)
for i in range(int(num_jobs)):
  job = jobsArray[i].strip().split()
  #print(job)
  # convert string to integer
  res = [eval(x) for x in job]
  #print(res)
  jobsDict2[i].append(res[0])
  jobsDict2[i].append(res[1])
  jobsDict2[i].append(res[0] / res[1])
print(jobsDict2)

In [ ]:
## I know that my dictionary has a list of values
def reverse_sort_dict_by_value(dict2):
    """Sorts a dictionary by value in descending order, breaking ties by one of the values in the list."""

    # Create a list of tuples, where each tuple contains a key-value pair.
    items = list(dict2.items())
    print("Unsorted dictionary")
    print(items)

    # Sort the list of tuples by value in descending order.
    items.sort(key=lambda x: x[1][2] , reverse=True)
    print("Sorted dictionary")
    print(items)

    return items

In [ ]:
# Applying jobsDict2
total_completion_time = 0
completion_time_so_far = 0
for item in reverse_sort_dict_by_value(jobsDict2):
  completion_time_so_far += item[1][1]
  total_completion_time += (completion_time_so_far*item[1][0])
print(total_completion_time)

Answer to part 2: 67311454237

# Programming Assignment : Part 3

In this programming problem you'll code up Prim's minimum spanning tree algorithm.

Download the text file below - edges.txt

This file describes an undirected graph with integer edge costs.  It has the format

[number_of_nodes] [number_of_edges]

[one_node_of_edge_1] [other_node_of_edge_1] [edge_1_cost]

[one_node_of_edge_2] [other_node_of_edge_2] [edge_2_cost]

...

For example, the third line of the file is "2 3 -8874", indicating that there is an edge connecting vertex #2 and vertex #3 that has cost -8874.

You should NOT assume that edge costs are positive, nor should you assume that they are distinct.

Your task is to run Prim's minimum spanning tree algorithm on this graph.  You should report the overall cost of a minimum spanning tree --- an integer, which may or may not be negative --- in the box below.

IMPLEMENTATION NOTES: This graph is small enough that the straightforward O(mn) time implementation of Prim's algorithm should work fine. OPTIONAL: For those of you seeking an additional challenge, try implementing a heap-based version. The simpler approach, which should already give you a healthy speed-up, is to maintain relevant edges in a heap (with keys = edge costs).  The superior approach stores the unprocessed vertices in the heap, as described in lecture.  Note this requires a heap that supports deletions, and you'll probably need to maintain some kind of mapping between vertices and their positions in the heap.

In [ ]:
df_prims = pd.read_table('https://d3c33hcgiwev3.cloudfront.net/_d4f3531eac1d289525141e95a2fea52f_edges.txt?Expires=1713744000&Signature=HlWEoAFTbqzHDA-jsxiaB31QLF9hCBiQcVVQ5MWksHTZpu1ON-SDod1Oi5oCDCXX2Q9fSo67j6KdgEn9TtgYB7HeehtiosEGdRUnOMSS-Jx1KNzBTC8JkDC-xwecyymyUoEe1z1fCI1ULXDeRexwK4kP0Rjityey0QUgPRgdNQw_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A', header=None)
df_prims.head()

In [ ]:
first_row = df_prims[0].iloc[0]
num_nodes, num_edges = [eval(x) for x in first_row.strip().split(' ')]
print(num_nodes, num_edges)

In [ ]:
df_prims = df_prims.drop(df_prims.index[0])
df_prims.head()

## MST Graph Class

In [ ]:
class Graph():

  def __init__(self, V):
    self.V = V
    self.graph = defaultdict(list)

  # Adds an edge to an undirected graph
  def addEdge(self, src, dest, weight):
    # Add an edge from src to dest.  A new node is
    # added to the adjacency list of src. The node
    # is added at the beginning. The first element of
    # the node has the destination and the second
    # elements has the weight
    newNode = [dest, weight]
    self.graph[src].insert(0, newNode)

    # Since graph is undirected, add an edge from
    # dest to src also
    newNode = [src, weight]
    self.graph[dest].insert(0, newNode)

  # The main function that prints the Minimum
  # Spanning Tree(MST) using the Prim's Algorithm.
  # It is a O(mLogn) function
  # The function calculates the cost of minimum spanning tree
  def primsMST(self):
    # Pick an arbitrary starting point
    start = next(iter(self.graph))
    explored, treeCost = set(), 0
    tiebreak = count().__next__ # Factory for tie-breaking values
    unexplored = [(0, tiebreak(), start)]

    while unexplored:
        cost, _, winner = heappop(unexplored)
        # If the node is already explored, just pop it and ignore it
        if winner in explored:
            continue

        explored.add(winner)
        treeCost += cost

        #adding the new crossing edges to heap
        for neighbor, cost in self.graph[winner]:
            if neighbor not in explored:
                heappush(unexplored, (cost, tiebreak(), neighbor))

    return treeCost

In [ ]:
EdgesArray = df_prims[0].to_list()
print(EdgesArray)

In [ ]:
graph = Graph(num_nodes)
for edge in EdgesArray:
  src, dest, weight = [eval(x) for x in edge.strip().split(' ')]
  graph.addEdge(src, dest, weight)

In [ ]:
print(graph.primsMST())

Answer for MST is -3612829

### Tests

In [ ]:
g2 = Graph(4)
g2.addEdge(1, 2, 1)
g2.addEdge(2, 4, 2)
g2.addEdge(3, 1, 4)
g2.addEdge(4, 3, 5)
g2.addEdge(4, 1, 3)
print(g2.primsMST())

In [ ]:
g3 = Graph(8)
g3.addEdge(8, 1, 4)
g3.addEdge(1, 2, 1)
g3.addEdge(1, 8, 8)
g3.addEdge(2, 8, 2)
g3.addEdge(8, 3, 6)
g3.addEdge(2, 3, 9)
g3.addEdge(8, 4, 9)
g3.addEdge(4, 3, 4)
g3.addEdge(8, 7, 2)
g3.addEdge(4, 7, 1)
g3.addEdge(4, 5, 2)
g3.addEdge(3, 5, 5)
g3.addEdge(7, 5, 7)
g3.addEdge(5, 6, 3)
g3.addEdge(7, 6, 10)

print(g3.primsMST())

In [ ]:
g4 = Graph(6)
g4.addEdge(1, 2, 6)
g4.addEdge(1, 4, 5)
g4.addEdge(1, 5, 4)
g4.addEdge(2, 4, 1)
g4.addEdge(2, 5, 2)
g4.addEdge(2, 3, 5)
g4.addEdge(2, 6, 3)
g4.addEdge(3, 6, 4)
g4.addEdge(4, 5, 2)
g4.addEdge(5, 6, 4)

print(g4.primsMST())